# Advanced LangChain Patterns

This notebook demonstrates advanced LangChain patterns that go beyond basic usage, positioning you as a thought leader in LangChain + Snowflake integration.

## What You'll Learn

1. **Router Patterns** - LLM-based decision making and workflow routing
2. **Tool-Calling Agents** - Compare LangChain vs LangGraph approaches for intelligent tool usage

## Prerequisites

- Completed `getting_started.ipynb` and `snowflake_workflows.ipynb`
- Understanding of LangChain basics (chat models, tools, chains)
- (Optional) Familiarity with LangGraph concepts

## Why These Patterns Matter

These patterns represent the cutting edge of LangChain application development:
- **Router patterns** enable dynamic workflow selection based on user intent
- **Tool-calling agents** provide intelligent, multi-step problem solving
- **Framework comparisons** help you choose the right approach for your use case


## 1. Router Patterns

Router patterns allow LLMs to make intelligent decisions about which workflow, tool, or processing path to take based on user input. This enables dynamic, context-aware applications.


In [ ]:
# Setup
from typing import Literal

from pydantic import BaseModel

from langchain_snowflake import ChatSnowflake, create_session_from_env
from langchain_snowflake.tools import (
    CortexSentimentTool,
    CortexSummarizerTool,
    SnowflakeCortexAnalyst,
)

# Initialize session and LLM - consistent with other notebooks
session = create_session_from_env()
llm = ChatSnowflake(
    session=session, model="claude-3-5-sonnet", temperature=0.1, max_tokens=1000
)

print("Session and LLM initialized successfully")

### 1.1 Basic Router - Workflow Selection

The simplest router pattern: decide between different workflows based on user intent.


In [ ]:
class WorkflowRoute(BaseModel):
    """Route for workflow selection."""

    workflow: Literal["analytics", "support", "content", "research"]
    confidence: float
    reasoning: str
    suggested_action: str


def create_workflow_router():
    """Create a workflow router using structured output."""

    # Use global session if available, otherwise create fresh one
    global session
    if session is None:
        print("Creating fresh session for workflow router...")
        router_session = create_session_from_env()
    else:
        print("Using existing session for workflow router...")
        router_session = session

    router_llm_base = ChatSnowflake(
        session=router_session,
        model="claude-3-5-sonnet",
        temperature=0.1,
        max_tokens=1000,
    )
    router_llm = router_llm_base.with_structured_output(WorkflowRoute)
    print("Router created successfully")

    def route_workflow(user_input: str) -> WorkflowRoute:
        prompt = f"""
        Analyze the user request and route to the appropriate workflow:
        
        User Request: "{user_input}"
        
        Available Workflows:
        - analytics: Data analysis, SQL queries, business intelligence, metrics
        - support: Help requests, troubleshooting, how-to questions  
        - content: Text processing, summarization, translation, writing
        - research: Information gathering, market research, competitive analysis
        
        Consider the user's intent and recommend the best workflow with high confidence.
        """

        return router_llm.invoke(prompt)

    return route_workflow


# Test the workflow router
router = create_workflow_router()

test_requests = [
    "Show me Q4 sales performance by region",
    "How do I reset my password?",
    "Summarize this 10-page document for me",
    "What are the latest trends in cloud computing?",
    "Compare sentiment analysis between our product reviews and competitors",
]

print("\nWorkflow Router Testing")

for i, request in enumerate(test_requests, 1):
    route = router(request)
    print(f'\nRequest {i}: "{request}"')

    # Handle both Pydantic model and dict responses
    if isinstance(route, dict):
        # Fallback: structured output returned as dict
        workflow = route.get("workflow", "unknown")
        confidence = route.get("confidence", 0.0)
        reasoning = route.get("reasoning", "No reasoning provided")
        action = route.get("suggested_action", "No action provided")
    else:
        # Expected: Pydantic model with attributes
        workflow = route.workflow
        confidence = route.confidence
        reasoning = route.reasoning
        action = route.suggested_action

    print(f"Route: {workflow} (confidence: {confidence:.2f})")
    print(f"Reasoning: {reasoning}")
    print(f"Action: {action}")

print("Basic workflow routing completed!")

### 1.2 Advanced Router - Conditional Execution

Execute different processing chains based on routing decisions:


In [ ]:
from langchain_snowflake import ChatSnowflake, create_session_from_env


def create_execution_router():
    """Create a router that executes different chains based on workflow type."""

    # Use global session if available, otherwise create fresh one
    global session
    if session is None:
        print("Creating fresh session for execution router...")
        exec_session = create_session_from_env()
    else:
        print("Using existing session for execution router...")
        exec_session = session

    exec_llm = ChatSnowflake(
        session=exec_session,
        model="claude-3-5-sonnet",
        temperature=0.1,
        max_tokens=1000,
    )
    print("Execution router session ready")

    # Initialize tools with session
    sentiment_tool = CortexSentimentTool(session=exec_session)
    summarizer_tool = CortexSummarizerTool(session=exec_session)
    analyst_tool = SnowflakeCortexAnalyst(session=exec_session)

    def analytics_chain(request: str) -> str:
        """Handle analytics requests."""
        try:
            # Generate SQL with Cortex Analyst
            sql_result = analyst_tool.run(request)

            # Analyze the result with LLM
            analysis_prompt = f"""
            Analyze this SQL generation result for the request: "{request}"
            
            SQL Result: {sql_result}
            
            Provide insights about what this query will accomplish and any recommendations.
            """

            analysis = exec_llm.invoke(analysis_prompt)
            return f"Analytics Result:\n{sql_result}\n\nAnalysis:\n{analysis.content}"
        except Exception as e:
            return f"Analytics error: {e}"

    def content_chain(request: str) -> str:
        """Handle content processing requests."""
        try:
            # First, determine content operation
            if "sentiment" in request.lower():
                # Extract text for sentiment analysis
                text_prompt = (
                    f"Extract the main text content from this request: {request}"
                )
                text_response = exec_llm.invoke(text_prompt)
                text = text_response.content

                sentiment_result = sentiment_tool.run(text)
                return f"Sentiment Analysis:\n{sentiment_result}"

            elif "summary" in request.lower() or "summarize" in request.lower():
                # Extract text for summarization
                text_prompt = (
                    f"Extract the main text content to summarize from: {request}"
                )
                text_response = exec_llm.invoke(text_prompt)
                text = text_response.content

                summary_result = summarizer_tool.run(text)
                return f"Summary:\n{summary_result}"

            else:
                # General content processing
                content_prompt = f"""
                Process this content request: "{request}"
                
                Provide appropriate text processing, writing assistance, or content guidance.
                """
                content_response = exec_llm.invoke(content_prompt)
                return f"Content Processing:\n{content_response.content}"

        except Exception as e:
            return f"Content processing error: {e}"

    def support_chain(request: str) -> str:
        """Handle support requests."""
        support_prompt = f"""
        Provide helpful support for this request: "{request}"
        
        Give clear, actionable guidance and troubleshooting steps if applicable.
        Focus on being helpful and solving the user's problem.
        """

        support_response = exec_llm.invoke(support_prompt)
        return f"Support Response:\n{support_response.content}"

    def research_chain(request: str) -> str:
        """Handle research requests."""
        research_prompt = f"""
        Provide comprehensive research insights for: "{request}"
        
        Include relevant background, current trends, key considerations, and actionable recommendations.
        Structure your response as a brief research report.
        """

        research_response = exec_llm.invoke(research_prompt)
        return f"Research Report:\n{research_response.content}"

    # Router execution function
    def execute_request(request: str) -> str:
        """Route and execute request based on workflow type."""

        # First, route the request
        route = router(request)

        # Handle both Pydantic model and dict responses
        if isinstance(route, dict):
            workflow = route.get("workflow", "unknown")
        else:
            workflow = route.workflow

        print(f"Routing '{request[:50]}...' to '{workflow}' workflow")

        # Execute appropriate chain based on route
        if workflow == "analytics":
            return analytics_chain(request)
        elif workflow == "content":
            return content_chain(request)
        elif workflow == "support":
            return support_chain(request)
        elif workflow == "research":
            return research_chain(request)
        else:
            return f"Unknown workflow: {workflow}"

    return execute_request


# Test the execution router
execution_router = create_execution_router()

test_executions = [
    "Show me top customers by revenue this quarter",
    "Analyze the sentiment of: 'This new feature is absolutely fantastic!'",
    "How do I configure SSL settings in Snowflake?",
    "What are the emerging trends in real-time data processing?",
]

print("\nExecution Router Testing")

for i, request in enumerate(test_executions, 1):
    print(f"Request {i}: {request}")

    try:
        result = execution_router(request)
        print(result)
    except Exception as e:
        print(f"Execution error: {e}")

print("\nAdvanced conditional routing completed!")

## 2. Tool-Calling Agents

Compare LangChain vs LangGraph approaches for intelligent tool usage and multi-step reasoning.


### 2.1 LangChain Approach - bind_tools with multi-tool planning

The LangChain approach uses `bind_tools` with manual execution for better multi-tool coordination:


In [ ]:
from langchain_core.tools import tool
from datetime import datetime
from langchain_snowflake import (
    CortexSentimentTool,
    CortexSummarizerTool,
    CortexTranslatorTool,
    SnowflakeCortexAnalyst,
    SnowflakeQueryTool,
)

@tool
def get_current_timestamp() -> str:
    """Get the current timestamp in a readable format."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

@tool  
def calculate_percentage_change(old_value: float, new_value: float) -> str:
    """Calculate percentage change between two values."""
    if old_value == 0:
        return "Cannot calculate percentage change from zero"
    
    change = ((new_value - old_value) / old_value) * 100
    return f"Change from {old_value} to {new_value}: {change:.2f}% {'increase' if change > 0 else 'decrease'}"

class LangChainToolAgent:
    """Tool-calling agent using LangChain's native bind_tools."""

    def __init__(self, session):
        self.session = session

        # Initialize Snowflake tools
        self.snowflake_tools = [
            CortexSentimentTool(session=session),
            CortexSummarizerTool(session=session),
            CortexTranslatorTool(session=session),
            SnowflakeCortexAnalyst(session=session),
            SnowflakeQueryTool(session=session),
        ]

        # General tools
        self.general_tools = [get_current_timestamp, calculate_percentage_change]

        # Combine all tools
        self.all_tools = self.snowflake_tools + self.general_tools

        # Create LLM with bound tools
        self.llm = ChatSnowflake(
            session=session, model="claude-4-sonnet", temperature=0.1, max_tokens=1500
        ).bind_tools(self.all_tools)

    def run(self, query: str) -> str:
        """Execute query using pure native LLM tool selection."""
        try:
            response = self.llm.invoke(query)

            if hasattr(response, "tool_calls") and response.tool_calls:
                tool_count = len(response.tool_calls)
                print(f"   LLM selected {tool_count} tool(s)")
                return self._execute_tool_calls(response)
            else:
                print(f"   No tools called - LLM provided direct response")
                return response.content

        except Exception as e:
            return f"Error: {e}"

    def _execute_tool_calls(self, response) -> str:
        """Execute the tools selected by the LLM."""
        tool_results = []
        
        for tool_call in response.tool_calls:
            tool_name = tool_call["name"]
            tool_args = tool_call["args"]

            # Find and execute the tool
            for tool in self.all_tools:
                if tool.name == tool_name:
                    try:
                        result = tool.invoke(tool_args)
                        tool_results.append(f"• {tool_name}: {result}")
                        print(f"      Executed {tool_name}")
                    except Exception as e:
                        tool_results.append(f"• {tool_name}: Error - {e}")
                        print(f"      Error in {tool_name}: {e}")
                    break

        # Return results
        if tool_results:
            return response.content + "\n\nTool Execution Results:\n" + "\n".join(tool_results)
        else:
            return response.content

# Create and test the agent
langchain_agent = LangChainToolAgent(session)

# Test queries that FORCE tool usage (these cannot be answered without tools)
langchain_test_queries = [
    "What time it is right now?",
    "find the percentage change from 100 to 150",
    "Analyze the sentiment of this text: 'I love this product!'",
    "Please translate 'Hello world' to Spanish and French",
    "Execute this query: SELECT 'Tools working!' as status, CURRENT_TIMESTAMP() as timestamp, Tell me the current timestamp and then do a sentiment analysis of: 'I like this movie!'",
]

print(f"Available tools: {len(langchain_agent.all_tools)} total")
print("Tool List:")
for tool in langchain_agent.all_tools:
    print(f"   - {tool.name}: {tool.description}")

for i, query in enumerate(langchain_test_queries, 1):
    print(f"Test {i}: {query}")
    
    try:
        result = langchain_agent.run(query)
        print(f"Response: {result}")
    except Exception as e:
        print(f"Error: {e}")

print("\nLangChain approach completed!")


### 2.2 LangGraph Approach - ToolNode with Multi-Step Reasoning

The LangGraph approach uses explicit state management and ToolNode for more complex workflows:


In [ ]:
try:
    from langchain_core.messages import AIMessage, HumanMessage
    from langgraph.graph import END, StateGraph
    from langgraph.graph.message import MessagesState
    from langgraph.prebuilt import ToolNode

    from langchain_snowflake import (
        CortexSentimentTool,
        CortexSummarizerTool,
        CortexTranslatorTool,
        SnowflakeCortexAnalyst,
        SnowflakeQueryTool,
    )

    LANGGRAPH_AVAILABLE = True
    print("LangGraph available")
except ImportError:
    LANGGRAPH_AVAILABLE = False
    print("LangGraph not available - install with: pip install langgraph")

if LANGGRAPH_AVAILABLE:

    class LangGraphToolAgent:
        """Tool-calling agent using LangGraph's ToolNode approach."""

        def __init__(self, session):
            self.session = session

            # Initialize tools (same as LangChain)
            self.snowflake_tools = [
                CortexSentimentTool(session=session),
                CortexSummarizerTool(session=session),
                CortexTranslatorTool(session=session),
                SnowflakeCortexAnalyst(session=session),
                SnowflakeQueryTool(session=session),
            ]

            self.general_tools = [get_current_timestamp, calculate_percentage_change]

            self.all_tools = self.snowflake_tools + self.general_tools

            # Create LLM with tools (no auto-execute - LangGraph handles this)
            self.llm = ChatSnowflake(
                session=session,
                model="claude-3-5-sonnet",
                temperature=0.1,
                max_tokens=1500,
            ).bind_tools(self.all_tools)

            # Build LangGraph workflow
            self.workflow = self._build_workflow()

        def _build_workflow(self):
            """Build the LangGraph workflow."""

            # Create tool node for executing tools
            tool_node = ToolNode(self.all_tools)

            def call_model(state: MessagesState):
                """Call the model to get tool calls or final response."""
                messages = state["messages"]
                response = self.llm.invoke(messages)
                return {"messages": [response]}

            def should_continue(state: MessagesState):
                """Decide whether to continue with tools or end."""
                messages = state["messages"]
                last_message = messages[-1]

                # If the last message has tool calls, continue to tools
                if hasattr(last_message, "tool_calls") and last_message.tool_calls:
                    return "tools"
                # Otherwise, end the conversation
                return END

            # Build the graph
            workflow = StateGraph(MessagesState)

            # Add nodes
            workflow.add_node("agent", call_model)
            workflow.add_node("tools", tool_node)

            # Add edges
            workflow.set_entry_point("agent")
            workflow.add_conditional_edges(
                "agent", should_continue, {"tools": "tools", END: END}
            )
            workflow.add_edge("tools", "agent")

            return workflow.compile()

        def run(self, query: str) -> str:
            """Execute query using LangGraph workflow."""
            try:
                initial_state = {"messages": [HumanMessage(content=query)]}

                # Run the workflow
                result = self.workflow.invoke(initial_state)

                # Get the final response
                final_message = result["messages"][-1]
                return final_message.content

            except Exception as e:
                return f"LangGraph agent error: {e}"

        async def arun(self, query: str) -> str:
            """Execute query asynchronously using LangGraph."""
            try:
                initial_state = {"messages": [HumanMessage(content=query)]}

                # Run the workflow asynchronously
                result = await self.workflow.ainvoke(initial_state)

                # Get the final response
                final_message = result["messages"][-1]
                return final_message.content

            except Exception as e:
                return f"LangGraph async agent error: {e}"

    # Test LangGraph approach
    print("\nLangGraph Tool-Calling Agent")

    langgraph_agent = LangGraphToolAgent(session)

    langgraph_test_queries = [
        "What's the current timestamp and analyze the sentiment of: 'Amazing results!'",
        "Calculate percentage change from 200 to 250, then summarize what this means",
        "Create SQL query for customer sales grouped by product and get the current time",
    ]

    print(f"🛠️ Available tools: {len(langgraph_agent.all_tools)} total")
    print("   • 5 Snowflake Cortex tools")
    print("   • 2 general-purpose tools")
    print("   • Explicit multi-step reasoning")
    print("   • State management with MessagesState")

    for i, query in enumerate(langgraph_test_queries, 1):
        print(f"LangGraph Query {i}: {query}")

        try:
            result = langgraph_agent.run(query)
            print(f"Response: {result}")
        except Exception as e:
            print(f"Error: {e}")

    print("\nLangGraph approach completed!")

else:
    print(
        "\nLangGraph section skipped - please install langgraph to test this approach"
    )

## Summary

Congratulations! You've mastered advanced LangChain patterns with Snowflake:

### What You've Accomplished

1. **Router Patterns**
   - Built intelligent workflow routing based on user intent
   - Implemented conditional execution chains
   - Created dynamic, context-aware applications

2. **Tool-Calling Agents**
   - Compared LangChain vs LangGraph approaches
   - Built multi-tool coordination systems
   - Understood when to use each framework

### Next Steps

1. **Production Deployment**
   - Implement proper error handling and retries
   - Add monitoring and logging for tool executions
   - Set up authentication and security

2. **Advanced Patterns**
   - Explore LangGraph's advanced features (subgraphs, human-in-the-loop)
   - Build multi-agent systems with state sharing
   - Implement custom tool execution strategies

3. **Snowflake Integration**
   - Create custom tools for your specific Snowflake data
   - Build semantic models for better Cortex Analyst results
   - Implement RAG with your organization's data

### Key Takeaways

- **Router patterns** enable intelligent application behavior
- **LangChain bind_tools** is perfect for simple, fast tool calling
- **LangGraph ToolNode** excels at complex, multi-step workflows
- **Snowflake Cortex** provides powerful AI capabilities for data-driven applications

**You're now equipped to build sophisticated LangChain applications with Snowflake!**
